In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
load_dotenv()

True

In [2]:
headers = {
    "Authorization": "Bearer " + os.environ['NOTION_TOKEN'],
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

In [3]:
def get_pages(DATABASE_ID, num_pages=None):

    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    payload = {"page_size": page_size}
    response = requests.post(url, json=payload, headers=headers)

    data = response.json()

    results = data["results"]
    while data["has_more"]:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
        response = requests.post(url, json=payload, headers=headers)
        data = response.json()
        results.extend(data["results"])

    return results

In [5]:
def add_time(start_date, added_min=30):
    added_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%S.%f%z") + timedelta(minutes=added_min)
    return added_date.strftime("%Y-%m-%dT%H:%M:%S.%f%z")

In [4]:
results = get_pages(os.environ["DATABASE_ID"])
df = pd.DataFrame(results)
df = df.sort_values(by=['last_edited_time'], ascending=False)
df.columns

Index(['object', 'id', 'created_time', 'last_edited_time', 'created_by',
       'last_edited_by', 'cover', 'icon', 'parent', 'archived', 'properties',
       'url', 'public_url'],
      dtype='object')

In [16]:
#get all line where col properties contains "start_date tobe able to add time"
for index, row in df.iterrows:
    if in "Postuler" in str(row.properties):
        print(index, str(row.properties))

,object,id,created_time,last_edited_time,created_by,last_edited_by,cover,icon,parent,archived,properties,url,public_url


In [8]:
properties = df.loc[20,"properties"]
" ".join([project_item["plain_text"] for project_item in properties["Nom"]["title"]])

'Envoyer mail à Julia pour entrer career fair'

In [8]:
properties = df.loc[21,"properties"]

properties

{'Importance': {'id': 'FN%3DI',
  'type': 'select',
  'select': {'id': 'N<<x', 'name': '🟠 Moyenne', 'color': 'yellow'}},
 'Sous-élément': {'id': 'NY%3A%7C',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Notes': {'id': 'WViK', 'type': 'rich_text', 'rich_text': []},
 'Date': {'id': '%5DCyE',
  'type': 'date',
  'date': {'start': '2024-01-11T16:30:00.000+01:00',
   'end': '2024-01-11T16:45:00.000+01:00',
   'time_zone': None}},
 'Projet': {'id': '%5Deky',
  'type': 'multi_select',
  'multi_select': [{'id': '020a25aa-1993-42a5-b266-fd17bfed0d77',
    'name': '📗 Doctorat',
    'color': 'green'},
   {'id': 'c79ad5b5-cce9-40e1-8e85-f7f025f2e278',
    'name': '🎙 Conférences',
    'color': 'yellow'}]},
 'État': {'id': 'cM%5D%7B',
  'type': 'status',
  'status': {'id': '49faa935-9b75-4383-8919-a7ca52cf2c36',
   'name': 'Terminé',
   'color': 'green'}},
 'élément parent': {'id': 'j%3FEp',
  'type': 'relation',
  'relation': [{'id': 'cf2e104d-e3a2-474b-bd7c-edfa1de35e6d'},
   {'i

In [11]:
for index, row in df.iterrows():
    try: 
        h = row["properties"]["Date"]["date"]["start"]
    except:
        h = row["properties"]
        break
h

{'Importance': {'id': 'FN%3DI',
  'type': 'select',
  'select': {'id': 'vX`b', 'name': '🔴 Forte', 'color': 'red'}},
 'Sous-élément': {'id': 'NY%3A%7C',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Notes': {'id': 'WViK', 'type': 'rich_text', 'rich_text': []},
 'Date': {'id': '%5DCyE', 'type': 'date', 'date': None},
 'Projet': {'id': '%5Deky',
  'type': 'multi_select',
  'multi_select': [{'id': 'c79ad5b5-cce9-40e1-8e85-f7f025f2e278',
    'name': '🎙 Conférences',
    'color': 'yellow'}]},
 'État': {'id': 'cM%5D%7B',
  'type': 'status',
  'status': {'id': '49faa935-9b75-4383-8919-a7ca52cf2c36',
   'name': 'Terminé',
   'color': 'green'}},
 'élément parent': {'id': 'j%3FEp',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Entreprise': {'id': 'wdDU', 'type': 'multi_select', 'multi_select': []},
 'Nom': {'id': 'title',
  'type': 'title',
  'title': [{'type': 'text',
    'text': {'content': 'Contacter ESCP pour possible thèse', 'link': None},
    'annotations

In [16]:
def get_parent_name(df, properties):
    parents_id = [parent["id"] for parent in properties["Sous-élément"]["relation"]]
    parents_name = ""
    for parent_id in parents_id:
        parents_name += " ".join([parent_row["plain_text"] for parent_row in df.loc[df['id'] == parent_id, "properties"].iloc[0]["Nom"]["title"]])
    return parents_name

def get_end_date(df, properties):
    start_date = avoid_none(row, ["properties", "Date", "date", "start"])
    if start_date == "" :
        return ""
    else:
        return row["properties"]["Date"]["date"]["end"] if row["properties"]["Date"]["date"]["end"] is not None else add_time(row["properties"]["Date"]["date"]["start"])
                                                                                                                                                                                                                                                      
# def avoid_none(row, path):
#     try:
#         for p in path:
#             row = row[p]
#         return row
#     except:
#         return ""
def properties_to_df(df):
    df_event = pd.DataFrame()
    df_event["name"] = df.apply(lambda row: " ".join([project_item["plain_text"] for project_item in row["properties"]["Nom"]["title"]]), axis=1)

        
    # Get the name
    df_event["name"] = df.apply(lambda row: " ".join([project_item["plain_text"] for project_item in row["properties"]["Nom"]["title"]]), axis=1)

    # Get the task's importance
    df_event["importance"] = df.apply(lambda row: avoid_none(row, ["properties", "Importance", "select", "name"]), axis=1)
    
    # Get the task's notes
    df_event["notes"] = df.apply(lambda row: " ".join([project_item["plain_text"] for project_item in row["properties"]["Notes"]["rich_text"]]), axis=1)  
    
    # Get dates
    df_event["start_date"] = df.apply(lambda row: datetime.fromisoformat(avoid_none(row, ["properties", "Date", "date", "start"])), axis=1)
    df_event["end_date"] = df.apply(lambda row: get_end_date(df, row["properties"]), axis=1)
    
    # Get the task's project
    df_event["project"] = df.apply(lambda row: " ".join([project_item["name"] for project_item in row["properties"]["Projet"]["multi_select"]]), axis=1)
    
    # Get parent_element 
    df_event["parents_name"] = df.apply(lambda row:get_parent_name(df, row["properties"]), axis=1)
    return df_event

In [ ]:
properties_to_df